# Predição de mortalidade em internações por pneumonias bacterianas sensíveis à Atenção Primária no Brasil, 2017-2021

## Análise preditiva

### 1. Importando pacotes de análise

In [1]:
# IMPORTANDO PACOTES 
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import pandas as pd  
import chart_studio.plotly as py
import plotly.graph_objs as gp 
import seaborn as sns

from scipy.stats import iqr
from sklearn.preprocessing import StandardScaler

### 2. Importando dados

In [2]:
#IMPOTANDO DADOS
sih_fn = pd.read_csv("sih_pneumonia_1721_trat.csv", index_col= 'N_AIH')
sih_fn

,IDADE,SEXO,MORTE,DIAS_PERM,DIAG_PRINC,MARCA_UTI,UTI_MES_TO,FAIXA_ETARIA,REGIAO
N_AIH,,,,,,,,,
1217100049066,74.0,1,1,8,1,0,0,1,0
1217100051343,63.0,1,0,1,1,0,0,1,0
1216100558520,74.0,0,0,8,1,0,0,1,0
1216100558937,32.0,0,1,7,1,0,0,0,0
1217100046118,1.0,0,0,5,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
1721101839105,75.0,1,0,18,0,0,0,1,0
1721101788659,66.0,0,0,13,1,0,0,1,0
1721101839644,81.0,1,0,9,0,0,0,1,0


### 3. Definindo teste e treino

In [3]:
#TAMANHO DA AMOSTRA DE TREINO
658719*0.7

461103.3

In [4]:
#SEPARANDO OS DADOS
x, y = sih_fn.drop("MORTE", axis= 1), sih_fn["MORTE"]
x_train, x_test, y_train, y_test = x[:461104], x[461104:], y[:461104], y[461104:]

In [5]:
#PRINTANDO AS CLASSES TARGET DO TREINO E TESTE
y_train.value_counts(), y_test.value_counts()

(0    418114
 1     42990
 Name: MORTE, dtype: int64,
 0    178207
 1     19408
 Name: MORTE, dtype: int64)

### 4. Testando algoritmos de classificação

#### 4.1 Regressão Logística

In [6]:
#IMPORTANDO PACOTE
from sklearn.linear_model import LogisticRegression

In [9]:
#DEFININDO FUNÇÃO PARA A ESTIMATIVA DO MODELO
softmax_reg = LogisticRegression(multi_class= "multinomial", solver = "lbfgs", C = 10)
modelo1 = softmax_reg.fit(x_train, y_train)

/home/daniellybx/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [10]:
#PREDIÇÕES DO MODELO
y_est = softmax_reg.predict(x_test)
pd.crosstab(y_test, y_est)

col_0,0,1
MORTE,,
0,176964,1243
1,18213,1195


##### 4.2 Gradiente Descendente Estocástico (SGD)

In [ ]:
#IMPORTANDO 